In [1]:
#Import dependencies
import pandas as pd

In [2]:
#Read in datasets
schools = pd.read_csv('raw_data/schools_complete.csv')
students = pd.read_csv('raw_data/students_complete.csv')

In [3]:
schools.head()

,School ID,name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [4]:
students.head()

,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [5]:
########## District Summary ##########

###########################################################################

#Find the Number of Students
n_schools = len(schools)
n_students = len(students)

print('# of Schools: ' + str(n_schools))
print('# of Students: ' + str(n_students))

###########################################################################

#Find the Total Budget
total_budget = schools['budget'].sum()
print('Total Budget: $' + str(total_budget))

###########################################################################

#Find Average Math and Reading Scores
math_score_avg = students['math_score'].mean()
read_score_avg = students['reading_score'].mean()

print('Avg Math Score: ' + str(math_score_avg))
print('Avg Reading Score: ' + str(read_score_avg))

###########################################################################

# Find the % of students passing Math, Reading, and Overall
# I'm making the cut-off to pass is >= 70 because a cut-off of 60 gives really high passing percentages
# and I wanted to see how the data looks with a lower percentage of passing students

perc_pass_math = len(students.loc[students['math_score'] >= 70]) / n_students * 100
perc_pass_read = len(students.loc[students['reading_score'] >= 70]) / n_students * 100

perc_pass_total = (perc_pass_math + perc_pass_read) / 2

print('% Passing Math: ' + str(round(perc_pass_math, 2)) + '%')
print('% Passing Reading: ' + str(round(perc_pass_read, 2)) + '%')
print('% Passing Overall: ' + str(round(perc_pass_total, 2)) + '%')

# of Schools: 15
# of Students: 39170
Total Budget: $24649428
Avg Math Score: 78.98537145774827
Avg Reading Score: 81.87784018381414
% Passing Math: 74.98%
% Passing Reading: 85.81%
% Passing Overall: 80.39%


In [6]:
#Summary Table
district_summary = pd.DataFrame({'Total Schools':[n_schools],
                                'Total Students':[n_students],
                                'Total Budget':[total_budget],
                                'Average Math Score':[math_score_avg],
                                'Average Reading Score':[read_score_avg],
                                '% Passing Math':[perc_pass_math],
                                '% Passing Reading':[perc_pass_read],
                                '% Passing Overall':[perc_pass_total]
                                })

###########################################################################

#Re-Order Columns
district_summary = district_summary[[
    'Total Schools',
    'Total Students',
    'Total Budget',
    'Average Math Score',
    'Average Reading Score',
    '% Passing Math',
    '% Passing Reading',
    '% Passing Overall'
]]

###########################################################################

#Round values to 4 decimal places
district_summary = district_summary.round(4)
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
0,15,39170,24649428,78.9854,81.8778,74.9809,85.8055,80.3932


In [7]:
#Create an unformatted dataframe for potential later use
district_summary_basic = district_summary

In [8]:
#Re-format District Summary Table
district_summary['Total Schools'] = district_summary['Total Schools'].map("{0:,.0f}".format)
district_summary['Total Students'] = district_summary['Total Students'].map("{0:,.0f}".format)
district_summary['Total Budget'] = district_summary['Total Budget'].map("${0:,.0f}".format)
district_summary['Average Math Score'] = district_summary['Average Math Score'].map("{0:,.2f}%".format)
district_summary['Average Reading Score'] = district_summary['Average Reading Score'].map("{0:,.2f}%".format)
district_summary['% Passing Math'] = district_summary['% Passing Math'].map("{0:,.2f}%".format)
district_summary['% Passing Reading'] = district_summary['% Passing Reading'].map("{0:,.2f}%".format)
district_summary['% Passing Overall'] = district_summary['% Passing Overall'].map("{0:,.2f}%".format)
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
0,15,"39,170","$24,649,428",78.99%,81.88%,74.98%,85.81%,80.39%


In [9]:
########## Schools Summary ##########

#* Create an overview table that summarizes key metrics about each school, including:
#  * School Name
#  * School Type
#  * Total Students
#  * Total School Budget
#  * Per Student Budget
#  * Average Math Score
#  * Average Reading Score
#  * % Passing Math
#  * % Passing Reading
#  * Overall Passing Rate (Average of the above two)

###########################################################################

# Already have most of the variables in the 'schools' data frame
# We need to calculate the Per Student Budget, average math/reading scores, and % Passing Rates...
# Then add those columns to the original 'schools' df, but rename to 'schools_summary

schools['budget_per_student'] = pd.to_numeric(schools['budget'] / schools['size'])

In [10]:
#Calculate Average Math/Reading Score by School
#Need to use the Students df, grouped by school

school_math_avg = pd.DataFrame(students.groupby('school')['math_score'].mean())
school_math_avg.reset_index(inplace = True)

school_read_avg = pd.DataFrame(students.groupby('school')['reading_score'].mean())
school_read_avg.reset_index(inplace = True)

In [11]:
#Calculate % of Students passing Math, Reading, and Overarll
#Add binary columns for pass_math, pass_read; 1 = pass, 0 = fail to 'temp' dataframe

#Create a temp table for the students dataframe
#temp_students = students
students['math_pass'] = 0
students['read_pass'] = 0

#students['total_pass'] = 0

#Reset Math/Read Pass variables based on Math/Read Scores
students.loc[students['math_score'] >= 70, 'math_pass'] = 1
students.loc[students['reading_score'] >= 70, 'read_pass'] = 1

#students.loc[(students['math_score'] >= 70) & (students['reading_score'] >= 70), 'total_pass'] = 1

In [12]:
# Group the tables by School on math_pass/read_pass
students_tot_math_pass = pd.DataFrame(students.groupby('school')['math_pass'].sum())
students_tot_math_pass.reset_index(inplace = True)

students_tot_read_pass = pd.DataFrame(students.groupby('school')['read_pass'].sum())
students_tot_read_pass.reset_index(inplace = True)

#students_tot_pass = pd.DataFrame(students.groupby('school')['total_pass'].sum())
#students_tot_pass.reset_index(inplace = True)

In [13]:
#Rename 'name' column to 'school' in 'school' dataframe
schools = schools.rename(columns={'name':'school'})

#Merge the Average Math & Reading Scores into one table
merged_df = pd.merge(school_math_avg, school_read_avg, on = 'school')

#Merge above with overall School Summary table
school_summary = pd.merge(schools, merged_df, on = 'school')

#Merge the Total # of Math Pass / Reading Pass / Total Pass tables with the overall School Summary Table
school_summary = pd.merge(school_summary, students_tot_math_pass, on = 'school')
school_summary = pd.merge(school_summary, students_tot_read_pass, on = 'school')

#school_summary = pd.merge(school_summary, students_tot_pass, on = 'school')

In [14]:
#Create columns for % of Math Pass / % of Read Pass
school_summary['budget_per_student'] = pd.to_numeric(school_summary['budget_per_student'])
school_summary['% Passing Math'] = pd.to_numeric(school_summary['math_pass'] / school_summary['size'] * 100)
school_summary['% Passing Reading'] = pd.to_numeric(school_summary['read_pass'] / school_summary['size'] * 100)
school_summary['% Overall Passing Rate'] = (school_summary['% Passing Math'] + school_summary['% Passing Reading']) / 2

school_summary = school_summary[['school', 'type', 'size', 'budget', 'budget_per_student', 'math_score', 'reading_score',
               '% Passing Math', '% Passing Reading', '% Overall Passing Rate']]

In [15]:
#Create an unformatted dataframe for later
school_summary_basic = school_summary
school_summary_basic.head()

,school,type,size,budget,budget_per_student,math_score,reading_score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
2,Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,93.867121,95.854628,94.860875
3,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,73.807983
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668


In [16]:
#Rename columns
school_summary = school_summary.rename(columns={'school':'School Name',
                                                'type':'School Type',
                                                'size':'Total Students',
                                                'budget':'Total School Budget',
                                                'budget_per_student':'Per Student Budget',
                                                'math_score':'Average Math Score',
                                                'reading_score':'Average Reading Score',
                                               })

#Round numeric columns to 4 decimal places
school_summary = school_summary.round(4)

In [17]:
#Reformat Columns
school_summary['Total Students'] = school_summary['Total Students'].map('{0:,.0f}'.format)
school_summary['Total School Budget'] = school_summary['Total School Budget'].map('${0:,.0f}'.format)
school_summary['Per Student Budget'] = school_summary['Per Student Budget'].map('${0:,.2f}'.format)
school_summary['Average Math Score'] = school_summary['Average Math Score'].map('{0:,.2f}'.format)
school_summary['Average Reading Score'] = school_summary['Average Reading Score'].map('{0:,.2f}'.format)
school_summary['% Passing Math'] = school_summary['% Passing Math'].map('{0:,.2f}%'.format)
school_summary['% Passing Reading'] = school_summary['% Passing Reading'].map('{0:,.2f}%'.format)
school_summary['% Overall Passing Rate'] = school_summary['% Overall Passing Rate'].map('{0:,.2f}%'.format)

In [18]:
#Sort from highest to lowest % Overall Passing  Rate
school_summary = school_summary.sort_values('% Overall Passing Rate', ascending = False)
#school_summary.reset_index(inplace = True)
school_summary

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
6,Cabrera High School,Charter,"1,858","$1,081,356",$582.00,83.06,83.98,94.13%,97.04%,95.59%
14,Thomas High School,Charter,"1,635","$1,043,130",$638.00,83.42,83.85,93.27%,97.31%,95.29%
4,Griffin High School,Charter,"1,468","$917,500",$625.00,83.35,83.82,93.39%,97.14%,95.27%
9,Pena High School,Charter,962,"$585,858",$609.00,83.84,84.04,94.59%,95.95%,95.27%
5,Wilson High School,Charter,"2,283","$1,319,574",$578.00,83.27,83.99,93.87%,96.54%,95.20%
10,Wright High School,Charter,"1,800","$1,049,400",$583.00,83.68,83.95,93.33%,96.61%,94.97%
2,Shelton High School,Charter,"1,761","$1,056,600",$600.00,83.36,83.73,93.87%,95.85%,94.86%
8,Holden High School,Charter,427,"$248,087",$581.00,83.80,83.81,92.51%,96.25%,94.38%
7,Bailey High School,District,"4,976","$3,124,928",$628.00,77.05,81.03,66.68%,81.93%,74.31%
3,Hernandez High School,District,"4,635","$3,022,020",$652.00,77.29,80.93,66.75%,80.86%,73.81%


In [19]:
#Create a Table of the Top 5 Performing Schools by Overall Passing Rate
schools_top5 = school_summary.head(5)
schools_top5

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
6,Cabrera High School,Charter,"1,858","$1,081,356",$582.00,83.06,83.98,94.13%,97.04%,95.59%
14,Thomas High School,Charter,"1,635","$1,043,130",$638.00,83.42,83.85,93.27%,97.31%,95.29%
4,Griffin High School,Charter,"1,468","$917,500",$625.00,83.35,83.82,93.39%,97.14%,95.27%
9,Pena High School,Charter,962,"$585,858",$609.00,83.84,84.04,94.59%,95.95%,95.27%
5,Wilson High School,Charter,"2,283","$1,319,574",$578.00,83.27,83.99,93.87%,96.54%,95.20%


In [20]:
#Create a Table of the Bottom 5 Performing Schools by Overall Passing Rate
schools_bottom5 = school_summary.tail(5)
schools_bottom5

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
13,Ford High School,District,"2,739","$1,763,916",$644.00,77.10,80.75,68.31%,79.30%,73.80%
12,Johnson High School,District,"4,761","$3,094,650",$650.00,77.07,80.97,66.06%,81.22%,73.64%
0,Huang High School,District,"2,917","$1,910,635",$655.00,76.63,81.18,65.68%,81.32%,73.50%
1,Figueroa High School,District,"2,949","$1,884,411",$639.00,76.71,81.16,65.99%,80.74%,73.36%
11,Rodriguez High School,District,"3,999","$2,547,363",$637.00,76.84,80.74,66.37%,80.22%,73.29%


In [21]:
# Math Scores by Grade**
# Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

# Create separate tables from the students table for each grade level
grade9 = students.loc[students['grade'] == '9th']
grade10 = students.loc[students['grade'] == '10th']
grade11 = students.loc[students['grade'] == '11th']
grade12 = students.loc[students['grade'] == '12th']

In [22]:
#Group datasets by school and calculate the average math_score
grade9_math = pd.DataFrame(grade9.groupby('school')['math_score'].mean())
grade9_math.reset_index(inplace = True)
grade9_math = grade9_math.rename(columns = {'math_score':'Grade 9 Math Score'})

grade10_math = pd.DataFrame(grade10.groupby('school')['math_score'].mean())
grade10_math.reset_index(inplace = True)
grade10_math = grade10_math.rename(columns = {'math_score':'Grade 10 Math Score'})

grade11_math = pd.DataFrame(grade11.groupby('school')['math_score'].mean())
grade11_math.reset_index(inplace = True)
grade11_math = grade11_math.rename(columns = {'math_score':'Grade 11 Math Score'})

grade12_math = pd.DataFrame(grade12.groupby('school')['math_score'].mean())
grade12_math.reset_index(inplace = True)
grade12_math = grade12_math.rename(columns = {'math_score':'Grade 12 Math Score'})


In [23]:
#Merge data sets on the 'school' variable
temp = pd.merge(grade9_math, grade10_math, on = 'school')
temp = pd.merge(temp, grade11_math, on = 'school')
temp = pd.merge(temp, grade12_math, on = 'school')

math_grades = temp.rename(columns = {'school':'Avg Math Scores',
                                     'Grade 9 Math Score':'Grade 9',
                                    'Grade 10 Math Score':'Grade 10',
                                    'Grade 11 Math Score':'Grade 11',
                                    'Grade 12 Math Score':'Grade 12'
                                    })

In [24]:
#Round columns to 4 decimal places
math_grades = math_grades.round(4)
math_grades

#Re-format Columns
math_grades['Grade 9'] = math_grades['Grade 9'].map('{0:,.2f}'.format)
math_grades['Grade 10'] = math_grades['Grade 10'].map('{0:,.2f}'.format)
math_grades['Grade 11'] = math_grades['Grade 11'].map('{0:,.2f}'.format)
math_grades['Grade 12'] = math_grades['Grade 12'].map('{0:,.2f}'.format)

math_grades

,Avg Math Scores,Grade 9,Grade 10,Grade 11,Grade 12
0,Bailey High School,77.08,77.00,77.52,76.49
1,Cabrera High School,83.09,83.15,82.77,83.28
2,Figueroa High School,76.40,76.54,76.88,77.15
3,Ford High School,77.36,77.67,76.92,76.18
4,Griffin High School,82.04,84.23,83.84,83.36
5,Hernandez High School,77.44,77.34,77.14,77.19
6,Holden High School,83.79,83.43,85.00,82.86
7,Huang High School,77.03,75.91,76.45,77.23
8,Johnson High School,77.19,76.69,77.49,76.86
9,Pena High School,83.63,83.37,84.33,84.12


In [25]:
# Reading Scores by Grade**
# Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

# Create separate tables from the students table for each grade level
grade9_read = students.loc[students['grade'] == '9th']
grade10_read = students.loc[students['grade'] == '10th']
grade11_read = students.loc[students['grade'] == '11th']
grade12_read = students.loc[students['grade'] == '12th']

grade9_read.head()

,Student ID,name,gender,grade,school,reading_score,math_score,math_pass,read_pass
0,0,Paul Bradley,M,9th,Huang High School,66,79,1,0
4,4,Bonnie Ray,F,9th,Huang High School,97,84,1,1
5,5,Bryan Miranda,M,9th,Huang High School,94,94,1,1
12,12,Brittney Walker,F,9th,Huang High School,64,79,1,0
13,13,William Long,M,9th,Huang High School,71,79,1,1


In [26]:
#Group datasets by school and calculate the average read_score
grade9_read = pd.DataFrame(grade9_read.groupby('school')['reading_score'].mean())
grade9_read.reset_index(inplace = True)
grade9_read = grade9_read.rename(columns = {'reading_score':'Grade 9'})

grade10_read = pd.DataFrame(grade10_read.groupby('school')['reading_score'].mean())
grade10_read.reset_index(inplace = True)
grade10_read = grade10_read.rename(columns = {'reading_score':'Grade 10'})

grade11_read = pd.DataFrame(grade11_read.groupby('school')['reading_score'].mean())
grade11_read.reset_index(inplace = True)
grade11_read = grade11_read.rename(columns = {'reading_score':'Grade 11'})

grade12_read = pd.DataFrame(grade12_read.groupby('school')['reading_score'].mean())
grade12_read.reset_index(inplace = True)
grade12_read = grade12_read.rename(columns = {'reading_score':'Grade 12'})

In [27]:
temp2 = pd.merge(grade9_read, grade10_read, on = 'school')
temp2 = pd.merge(temp2, grade11_read, on = 'school')
temp2 = pd.merge(temp2, grade12_read, on = 'school')

reading_grades = temp2.rename(columns = {'school':'Avg Reading Scores'})
reading_grades

,Avg Reading Scores,Grade 9,Grade 10,Grade 11,Grade 12
0,Bailey High School,81.303155,80.907183,80.945643,80.912451
1,Cabrera High School,83.676136,84.253219,83.788382,84.287958
2,Figueroa High School,81.198598,81.408912,80.640339,81.384863
3,Ford High School,80.632653,81.262712,80.403642,80.662338
4,Griffin High School,83.369193,83.706897,84.288089,84.013699
5,Hernandez High School,80.866860,80.660147,81.396140,80.857143
6,Holden High School,83.677165,83.324561,83.815534,84.698795
7,Huang High School,81.290284,81.512386,81.417476,80.305983
8,Johnson High School,81.260714,80.773431,80.616027,81.227564
9,Pena High School,83.807273,83.612000,84.335938,84.591160


In [28]:
#Round columns to 4 decimal places
reading_grades = reading_grades.round(4)
reading_grades

#Re-format Columns
reading_grades['Grade 9'] = reading_grades['Grade 9'].map('{0:,.2f}'.format)
reading_grades['Grade 10'] = reading_grades['Grade 10'].map('{0:,.2f}'.format)
reading_grades['Grade 11'] = reading_grades['Grade 11'].map('{0:,.2f}'.format)
reading_grades['Grade 12'] = reading_grades['Grade 12'].map('{0:,.2f}'.format)

reading_grades

,Avg Reading Scores,Grade 9,Grade 10,Grade 11,Grade 12
0,Bailey High School,81.30,80.91,80.95,80.91
1,Cabrera High School,83.68,84.25,83.79,84.29
2,Figueroa High School,81.20,81.41,80.64,81.38
3,Ford High School,80.63,81.26,80.40,80.66
4,Griffin High School,83.37,83.71,84.29,84.01
5,Hernandez High School,80.87,80.66,81.40,80.86
6,Holden High School,83.68,83.32,83.82,84.70
7,Huang High School,81.29,81.51,81.42,80.31
8,Johnson High School,81.26,80.77,80.62,81.23
9,Pena High School,83.81,83.61,84.34,84.59


In [29]:
#**Scores by School Spending**

#* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
#  * Average Math Score
#  * Average Reading Score
#  * % Passing Math
#  * % Passing Reading
#  * Overall Passing Rate (Average of the above two)


In [30]:
#Recreate school_summary without formatting -- otherwise the binning doesn't work

In [31]:
#See the min/max budget per student of the school_summary table to see what reasonable ranges are
print('Min: ' + str(school_summary_basic['budget_per_student'].min()))
print('Max: ' + str(school_summary_basic['budget_per_student'].max()))

Min: 578.0
Max: 655.0


In [32]:
#Create bins
bins = [0, 585, 615, 645, 675]
group_labels = ['$0-$585', '$585-$615','$615-$645','$645-$675']

In [33]:
# Place the data series into a new column inside of the DataFrame
school_summary_bins = school_summary_basic
school_summary_bins['budget_category'] = pd.cut(school_summary_basic['budget_per_student'], bins, labels = group_labels)
school_summary_bins.head()

,school,type,size,budget,budget_per_student,math_score,reading_score,% Passing Math,% Passing Reading,% Overall Passing Rate,budget_category
0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171,$645-$675
1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852,$615-$645
2,Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,93.867121,95.854628,94.860875,$585-$615
3,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,73.807983,$645-$675
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668,$615-$645


In [34]:
#Create new data frame
school_spending = school_summary_bins[['budget_category', 
                                       'math_score',
                                       'reading_score',
                                       '% Passing Math',
                                       '% Passing Reading',
                                       '% Overall Passing Rate']]

#Rename columns
school_spending = school_spending.rename(columns = {'budget_category':'Budget Range',
                                     'math_score':'Avg Math Score',
                                    'reading_score':'Avg Reading Score'
                                    })

In [35]:
# Create a GroupBy object based upon "budget_category"
school_spending = school_spending.groupby('Budget Range')
school_spending.mean()

,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Budget Range,,,,,
$0-$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-$615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-$645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-$675,76.997210,81.027843,66.164813,81.133951,73.649382


In [36]:
#Re-format dataframe
#school_spending['Avg Math Score'] = school_spending['Avg Math Score'].map('{0:,.2f}'.format)
#school_spending['Avg Reading Score'] = school_spending['Avg Reading Score'].map('{0:,.2f}'.format)
#school_spendings['Passing Math'] = school_spending['% Passing Math'].map('{0:,.2f}%'.format)
#school_spending['% Passing school_spendingading'] = school_spending['% Passing Reading'].map('{0:,.2f}%'.format)
#school_spending['% Overall Passing Rate'] = school_spending['% Overall Passing Rate'].map('{0:,.2f}%'.format)

#scool_spending

In [37]:
#**Scores by School Size**

#* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
#  * Average Math Score
#  * Average Reading Score
#  * % Passing Math
#  * % Passing Reading
#  * Overall Passing Rate (Average of the above two)

#See the min/max budget per student of the school_summary table to see what reasonable ranges are
print('Min: ' + str(school_summary_basic['size'].min()))
print('Max: ' + str(school_summary_basic['size'].max()))

#Create bins
bins = [0, 1500, 3000, 4500, 6000]
group_labels = ['0-1500', '1500-3000','3000-4500','4500-6000']

# Place the data series into a new column inside of the DataFrame
school_summary_size = school_summary_basic
school_summary_size['size_bins'] = pd.cut(school_summary_basic['size'], bins, labels = group_labels)
school_summary_size.head()

Min: 427
Max: 4976


,school,type,size,budget,budget_per_student,math_score,reading_score,% Passing Math,% Passing Reading,% Overall Passing Rate,budget_category,size_bins
0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171,$645-$675,1500-3000
1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852,$615-$645,1500-3000
2,Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,93.867121,95.854628,94.860875,$585-$615,1500-3000
3,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,73.807983,$645-$675,4500-6000
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668,$615-$645,0-1500


In [38]:
#Create new data frame
school_size = school_summary_size[['size_bins', 
                                       'math_score',
                                       'reading_score',
                                       '% Passing Math',
                                       '% Passing Reading',
                                       '% Overall Passing Rate']]

#Rename columns
school_size = school_size.rename(columns = {'size_bins':'Size Range',
                                     'math_score':'Avg Math Score',
                                    'reading_score':'Avg Reading Score'
                                    })

In [39]:
# Create a GroupBy object based upon "Size Range"
school_size = school_size.groupby('Size Range')
school_size.mean()

,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Size Range,,,,,
0-1500,83.664898,83.892148,93.497607,96.445946,94.971776
1500-3000,80.904987,82.822740,83.556977,90.588593,87.072785
3000-4500,76.842711,80.744686,66.366592,80.220055,73.293323
4500-6000,77.136883,80.978256,66.496861,81.339570,73.918215


In [40]:
school_summary_basic.head()

,school,type,size,budget,budget_per_student,math_score,reading_score,% Passing Math,% Passing Reading,% Overall Passing Rate,budget_category,size_bins
0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171,$645-$675,1500-3000
1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852,$615-$645,1500-3000
2,Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,93.867121,95.854628,94.860875,$585-$615,1500-3000
3,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,73.807983,$645-$675,4500-6000
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668,$615-$645,0-1500


In [44]:
#**Scores by School Type**

#* Create a table that breaks down school performances based on School Type (District, Charter).
#  Include in the table each of the following:
#  * Average Math Score
#  * Average Reading Score
#  * % Passing Math
#  * % Passing Reading
#  * Overall Passing Rate (Average of the above two)

#Create new data frame
school_type = school_summary_basic[['type', 'size', 'budget_per_student',
                                       'math_score',
                                       'reading_score',
                                       '% Passing Math',
                                       '% Passing Reading',
                                       '% Overall Passing Rate']]

#Rename columns
school_type = school_type.rename(columns = {'type':'School Type', 'size':'School Size',
                                            'budget_per_student':'Per Student Budget',
                                     'math_score':'Avg Math Score',
                                    'reading_score':'Avg Reading Score'
                                    })

In [45]:
# Create a GroupBy object based on "School Type"
school_type = school_type.groupby('School Type')
school_type.mean()

,School Size,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,,,
Charter,1524.250000,599.500000,83.473852,83.896421,93.620830,96.586489,95.103660
District,3853.714286,643.571429,76.956733,80.966636,66.548453,80.799062,73.673757


In [ ]:
##### Three Observable Trends #####

# 1.a) Of the 15 schools in the district, the top 8 by % Overall Passing Rate are the Charter Schools, while the bottom 7 are all District Schools
# 1.b) Passing rates are significantly higher overall for Charter Schools vs. District Schools.
#      No charter school has an overall passing rate lower than 94%, while no district school has a passing rate above 74%

# 2.0) Charter Schools have almost twice as many students as the district schools on average

# 3.0) However, the average budget per student is over $43 higher at district schools compared to charter schools.
#      As such, it appears that there is no relationship between student performance and the school's budget in this data set.
